<a href="https://colab.research.google.com/github/rishubhkhurana/nlp/blob/main/S2S/Seq2SeqSimple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Book Keeping

In [1]:
!python -m spacy download de > /dev/null 2>&1
!python -m spacy download fr > /dev/null 2>&1
!python -m spacy download en > /dev/null 2>&1

## Importing 

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
from torchtext.data import Field, BucketIterator 
import spacy
from torchtext.datasets import Multi30k
import random
import numpy as np
from functools import partial
import time, math

## Seed everything

In [4]:
SEED = 111
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.cuda.manual_seed(SEED)

## Dataset

In [5]:
src_tokenizer = spacy.load('en')
tgt_tokenizer = spacy.load('de')

In [6]:
def tokenize(text,tokenizer = None, reverse=False):
    tokens = [tok.text for tok in tokenizer.tokenizer(text)]
    return tokens[::-1] if reverse else tokens

In [7]:
src_field = Field(init_token = '<sos>', eos_token = '<eos>', batch_first = True, tokenize = partial(tokenize, tokenizer = src_tokenizer, reverse=True), lower = True)
tgt_field = Field(init_token = '<sos>', eos_token = '<eos>', batch_first = True, tokenize = partial(tokenize, tokenizer = tgt_tokenizer, reverse=False), lower = True)

In [8]:
train_dset, valid_dset, test_dset = Multi30k.splits(exts = ('.en','.de'), fields = (src_field, tgt_field))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 645kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 169kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 164kB/s]


In [9]:
' '.join(vars(train_dset.examples[0])['src']),' '.join(vars(train_dset.examples[0])['trg'])

('. bushes many near outside are males white , young two',
 'zwei junge weiße männer sind im freien in der nähe vieler büsche .')

## Build vocab and create dataloaders

In [10]:
src_field.build_vocab(train_dset, min_freq = 2)
tgt_field.build_vocab(train_dset, min_freq = 2)

In [11]:
len(src_field.vocab), len(tgt_field.vocab)

(5893, 7855)

In [12]:
src_field.vocab.freqs.values()

dict_values([27623, 19, 222, 571, 871, 3717, 40, 2238, 3963, 2280, 3886, 6, 3, 45, 49165, 22, 146, 123, 14886, 1759, 404, 3, 175, 711, 193, 1723, 916, 183, 60, 60, 8035, 1614, 7525, 2324, 1880, 7781, 357, 105, 23, 10955, 2927, 1969, 13, 660, 1947, 368, 322, 926, 7, 32, 353, 1449, 1404, 4, 5, 144, 1263, 382, 2, 27, 1046, 1378, 52, 843, 6179, 3973, 137, 6871, 272, 27, 183, 364, 2, 491, 440, 173, 55, 12, 30, 416, 30, 1092, 2623, 1057, 108, 405, 51, 1675, 13, 7379, 2019, 25, 68, 192, 526, 22, 445, 52, 988, 56, 1, 1, 2, 3, 306, 380, 15, 65, 1274, 1743, 16, 20, 83, 3128, 310, 31, 86, 33, 3122, 121, 28, 405, 19, 178, 851, 361, 1, 725, 363, 190, 60, 4, 150, 16, 339, 66, 72, 5, 19, 447, 522, 245, 37, 168, 214, 5, 671, 301, 768, 314, 86, 724, 118, 17, 59, 124, 13, 534, 160, 228, 643, 752, 306, 83, 21, 1755, 52, 894, 1, 1, 117, 894, 6, 30, 4, 24, 380, 9, 565, 785, 66, 66, 2, 460, 76, 41, 44, 76, 173, 309, 38, 107, 2, 664, 492, 4, 120, 449, 584, 132, 2861, 14, 7, 53, 179, 1, 20, 76, 24, 31, 16, 3,

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [14]:
train_dl,valid_dl, test_dl  = BucketIterator.splits((train_dset,valid_dset,test_dset), batch_size = 128, device=device)

In [15]:
batch = next(iter(train_dl))

In [16]:
batch.src

tensor([[  2,   5, 406,  ...,   1,   1,   1],
        [  2,   5, 432,  ...,   1,   1,   1],
        [  2,   5, 141,  ...,   1,   1,   1],
        ...,
        [  2, 512,  54,  ...,   1,   1,   1],
        [  2,   5, 689,  ...,   1,   1,   1],
        [  2,   5, 220,  ...,   1,   1,   1]], device='cuda:0')

In [17]:
src_field.vocab.stoi['<pad>']

1

## Model building

In [18]:
class Encoder(nn.Module):
    def __init__(self,emb_dims=100, hidden_dims=256, dropout= 0.1, num_layers = 2, vocab_size=len(src_field.vocab), bidirectional=False):
        super().__init__()
        self.embedder = nn.Embedding(vocab_size, emb_dims)
        self.dropout = nn.Dropout(dropout)
        self.rnn = nn.LSTM(emb_dims, hidden_dims, dropout=dropout, num_layers=num_layers, bidirectional=bidirectional, batch_first=True)
    def forward(self, text):
        # text: [B, Seqlength]
        embedded = self.embedder(text)
        embedded = self.dropout(embedded)
        # embedded: [B, Seqlength, emb_dims]
        out,(hidden,cell) = self.rnn(embedded)
        # hidden: [num_layers*num_directions, batch_size, hidden_dims]
        # output: [B, Seq length, hidden_dims]

        return hidden,cell

In [19]:
class Decoder(nn.Module):
    def __init__(self, emb_dims=100, hidden_dims=256, dropout=0.1, num_layers = 2, vocab_size= len(tgt_field.vocab)):
        super().__init__()
        self.embedder =  nn.Embedding(vocab_size, emb_dims)
        self.dropout = nn.Dropout(dropout)
        self.rnn = nn.LSTM(emb_dims, hidden_dims, dropout=dropout, num_layers=num_layers, batch_first=True)
        self.classifier = nn.Linear(hidden_dims, vocab_size)
        self.vocab_size = vocab_size
    def forward(self, text,hidden,cell):
        #text: [B]
        embedded = self.embedder(text.unsqueeze(1))
        embedded = self.dropout(embedded)
        # embedded: [B,1,embed_dims]
        output,(hidden,cell) = self.rnn(embedded,(hidden,cell))
        # output: [B,1,hidden_dims]
        # hidden,cell: [num_layers,B,hidden_dims]
        output = self.classifier(output.squeeze(1))
        return output,hidden,cell



In [20]:
class Seq2SeqModel(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, src, tgt, tforce_ratio = 0.5):
        tgt_len = tgt.shape[1]
        bs = tgt.shape[0]
        device =  tgt.device
        vocab_size =  self.decoder.vocab_size
        #src: [B, seq]
        h,c = self.encoder(src)
        # h,c: [num_layers, B, hidden_dims]
        # create empty tensor to save targets
        output = torch.zeros((bs,tgt_len,vocab_size)).to(device)
        input = tgt[:,0]
        for i in range(1,tgt_len):
            out,h,c = self.decoder(input, h,c)
            output[:,i,:]  = out
            input = tgt[:,i] if random.random()<tforce_ratio else out.argmax(dim=-1) 
        
        return output



## Training Model

In [21]:
encoder_params = dict()
encoder_params['hidden_dims'] = 512
encoder_params['emb_dims'] = 256
encoder_params['num_layers'] = 3
encoder_params['dropout'] = 0.5

In [22]:
encoder = Encoder(**encoder_params).to(device)

In [23]:
decoder_params = dict()
decoder_params['hidden_dims'] = 512
decoder_params['emb_dims'] = 256
decoder_params['num_layers'] = 3
decoder_params['dropout'] = 0.5

In [24]:
decoder = Decoder(**decoder_params).to(device)

In [25]:


model = Seq2SeqModel(encoder,decoder).to(device)

In [26]:
batch = next(iter(train_dl))

In [27]:
def init_weight(model):
    for param in model.parameters():
        nn.init.uniform_(param.data,-0.08,0.08)
model.apply(init_weight)

Seq2SeqModel(
  (encoder): Encoder(
    (embedder): Embedding(5893, 256)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): LSTM(256, 512, num_layers=3, batch_first=True, dropout=0.5)
  )
  (decoder): Decoder(
    (embedder): Embedding(7855, 256)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): LSTM(256, 512, num_layers=3, batch_first=True, dropout=0.5)
    (classifier): Linear(in_features=512, out_features=7855, bias=True)
  )
)

In [28]:
def compute_params(model):
    all_params=0
    for param in model.parameters():
        all_params+=param.numel() if param.requires_grad else 0
    return all_params

In [29]:
print(f"Number of model parameters: {compute_params(model):,}")

Number of model parameters: 19,108,015


In [30]:
opt = optim.Adam(model.parameters())

In [31]:
criterion = nn.CrossEntropyLoss(ignore_index=tgt_field.vocab.stoi[tgt_field.pad_token])

In [32]:
criterion

CrossEntropyLoss()

In [33]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        #trg = [batch size, trg len]
        #output = [batch size, trg len,  output dim]
        
        output_dim = output.shape[-1]
        output = output[:,1:,:].contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)

        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [34]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[:,1:,:].contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [35]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [36]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_dl, opt, criterion, CLIP)
    valid_loss = evaluate(model, valid_dl, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 40s
	Train Loss: 5.227 | Train PPL: 186.186
	 Val. Loss: 5.053 |  Val. PPL: 156.439
Epoch: 02 | Time: 0m 40s
	Train Loss: 4.751 | Train PPL: 115.725
	 Val. Loss: 4.857 |  Val. PPL: 128.620
Epoch: 03 | Time: 0m 40s
	Train Loss: 4.360 | Train PPL:  78.226
	 Val. Loss: 4.649 |  Val. PPL: 104.456
Epoch: 04 | Time: 0m 40s
	Train Loss: 4.096 | Train PPL:  60.109
	 Val. Loss: 4.563 |  Val. PPL:  95.863
Epoch: 05 | Time: 0m 40s
	Train Loss: 3.926 | Train PPL:  50.724
	 Val. Loss: 4.420 |  Val. PPL:  83.136
Epoch: 06 | Time: 0m 40s
	Train Loss: 3.798 | Train PPL:  44.596
	 Val. Loss: 4.277 |  Val. PPL:  72.011
Epoch: 07 | Time: 0m 40s
	Train Loss: 3.660 | Train PPL:  38.880
	 Val. Loss: 4.248 |  Val. PPL:  69.972
Epoch: 08 | Time: 0m 40s
	Train Loss: 3.537 | Train PPL:  34.353
	 Val. Loss: 4.156 |  Val. PPL:  63.802
Epoch: 09 | Time: 0m 40s
	Train Loss: 3.407 | Train PPL:  30.178
	 Val. Loss: 4.132 |  Val. PPL:  62.276
Epoch: 10 | Time: 0m 40s
	Train Loss: 3.306 | Train PPL

In [39]:
test_loss = evaluate(model,test_dl, criterion)
    

In [40]:
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.986 | Test PPL:  53.818 |
